# GEOCODING & DISTANCE MEASUREMENT WITH GEOPY

# NOTES

### Not all names and addresses will be recognized.

# DOCUMENTATION

In [45]:
from IPython.display import IFrame
documentation = IFrame(src='https://geopy.readthedocs.io/en/latest/', width=1000, height=500)
display(documentation)

# IMPORTS

In [1]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance

import pandas as pd
from ipyleaflet import Map, AntPath, MeasureControl
import ipywidgets
from vega_datasets import data as vds

# GEOLOCATION DATA FROM ADDRESS
### Important: Change user_agent To Your Custom user_agent Name

In [47]:
# address or name
indy_500_address = '4790 W 16th St, Indianapolis, IN 46222'

# Nominatim geocoder for OpenStreetMap data
# *** change user_agent to your custom user_agent name ***
# geocode returns a location point by address
indy_500 = Nominatim(user_agent='tutorial').geocode(indy_500_address)
indy_500

Location(4790, West 16th Street, Speedway, Marion, Indiana, 46222, United States of America, (39.790318600000006, -86.23369319174797, 0.0))

### FULL ADDRESS

In [48]:
indy_500.address

'4790, West 16th Street, Speedway, Marion, Indiana, 46222, United States of America'

### LATITUDE AND LONGITUDE

In [49]:
(indy_500.latitude, indy_500.longitude)

(39.790318600000006, -86.23369319174797)

### ALL GEOLOCATION DATA

In [50]:
indy_500.raw

{'place_id': 100103405,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 51315962,
 'boundingbox': ['39.7899432', '39.7906845', '-86.2341062', '-86.2332791'],
 'lat': '39.790318600000006',
 'lon': '-86.23369319174797',
 'display_name': '4790, West 16th Street, Speedway, Marion, Indiana, 46222, United States of America',
 'class': 'tourism',
 'type': 'museum',
 'importance': 0.9590557430834509,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/tourist_museum.p.20.png'}

# GEOLOCATION DATA FROM LIST OF ADDRESSES
five tallest buildings csv data source (as of July 2020): Wikipedia

some building names may need to be changed to return an address

In [51]:
data = 'https://raw.githubusercontent.com/groundhogday321/dataframe-datasets/master/five_tallest_buildings.csv'
tallest_buildings = pd.read_csv(data)
tallest_buildings

,Building,City,Country,Height,Floors,Year
0,Burj Khalifa,Dubai,United Arab Emirates,2717,163,2010
1,Shanghai Tower,Shanghai,China,2073,128,2015
2,Makkah Royal Clock Tower,Mecca,Saudi Arabia,1971,120,2012
3,Ping An Finance Center,Shenzhen,China,1965,115,2017
4,Lotte World Tower,Seoul,South Korea,1819,123,2016


### RateLimiter and AsyncRateLimiter help perform bulk operations while gracefully handling error responses and adding delays when needed.

Nominatim usage policy requirement is maximum of 1 request per second

https://operations.osmfoundation.org/policies/nominatim/

In [52]:
# Nominatim geocoder for OpenStreetMap data with RateLimiter
geocoder = RateLimiter(Nominatim(user_agent='tutorial').geocode, min_delay_seconds=1)
tallest_buildings['Location'] = tallest_buildings['Building'].apply(geocoder)

# add latitude and longitude to dataframe
tallest_buildings['Latitude'] = tallest_buildings['Location'].apply(lambda loc: loc.latitude if loc else None)
tallest_buildings['Longitude'] = tallest_buildings['Location'].apply(lambda loc: loc.longitude if loc else None)

tallest_buildings

,Building,City,Country,Height,Floors,Year,Location,Latitude,Longitude
0,Burj Khalifa,Dubai,United Arab Emirates,2717,163,2010,"(برج خليفة, شارع الشيخ محمد بن راشد, Al Manzil...",25.197031,55.274221
1,Shanghai Tower,Shanghai,China,2073,128,2015,"(501, 银城中路, 洋泾街道, 浦东新区, 上海市, 200010, China 中国,...",31.235646,121.501266
2,Makkah Royal Clock Tower,Mecca,Saudi Arabia,1971,120,2012,"(Fairmont Makkah Clock Royal Tower, Bab Al Mal...",21.418184,39.825565
3,Ping An Finance Center,Shenzhen,China,1965,115,2017,"(5033, 益田路, 皇岗, 车公庙, 福田区, 广东省, 518048, China 中...",22.536724,114.050355
4,Lotte World Tower,Seoul,South Korea,1819,123,2016,"(롯데월드타워, 올림픽로, 송파구, 방이2동, 05551, 대한민국, (37.512...",37.512684,127.102629


# ADDRESS FROM LATITUDE AND LONGITUDE

In [53]:
# address from latitude and longitude
airports = vds.airports()
airports.head()

,iata,name,city,state,country,latitude,longitude
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [54]:
# Nominatim geocoder for OpenStreetMap data
# reverse returns an address by location point (latitude, longitude)
airport = Nominatim(user_agent='tutorial').reverse((airports.latitude[0], airports.longitude[0]))
airport

Location(Thigpen Field, Airport Road, Bay Springs, Jasper County, Mississippi, 39422, United States of America, (31.953765, -89.2345048, 0.0))

### FULL ADDRESS

In [55]:
airport.address

'Thigpen Field, Airport Road, Bay Springs, Jasper County, Mississippi, 39422, United States of America'

### ALL GEOLOCATION DATA

In [56]:
airport.raw

{'place_id': 3317657,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 368386020,
 'lat': '31.953765',
 'lon': '-89.2345048',
 'display_name': 'Thigpen Field, Airport Road, Bay Springs, Jasper County, Mississippi, 39422, United States of America',
 'address': {'aeroway': 'Thigpen Field',
  'road': 'Airport Road',
  'village': 'Bay Springs',
  'county': 'Jasper County',
  'state': 'Mississippi',
  'postcode': '39422',
  'country': 'United States of America',
  'country_code': 'us'},
 'boundingbox': ['31.938765', '31.968765', '-89.2495048', '-89.2195048']}

# EXAMPLE WITH BING USING API KEY

### LATITUDE AND LONGITUDE

Most API keys require registration with the service/site. Some are free and some are paid services. See documentation for services/sites and visit each site for details.

In [57]:
import PYTHON_API_KEYS

# API key
# *** use your API key here ***
# put in your Bing API key as string
bing = PYTHON_API_KEYS.API_KEYS.BING_MAPS

# geocoder using the Bing maps locations API
sd_zoo = geopy.geocoders.Bing(api_key=bing).geocode('2920 Zoo Dr, San Diego, CA 92101')
print(sd_zoo.latitude, sd_zoo.longitude)

32.735234 -117.149126


# CALCULATING DISTANCE

### BASIC EXAMPLE

In [2]:
# put in start latitude, longitude and end latitude, longitude
# geopy.distance.distance currently uses geodesic (source: documentation)
distance.distance((31.953765, -89.234505), (30.685861, -95.017928)).miles

352.98342452610046

In [4]:
distance.geodesic((31.953765, -89.234505), (30.685861, -95.017928)).miles

352.98342452610046

In [5]:
distance.great_circle((31.953765, -89.234505), (30.685861, -95.017928)).miles

352.37625288490705

### EXAMPLE WITH MAP
### see ipywidgets and ipyleaflet tutorials for more examples

In [59]:
# widgets
start_dropdown = ipywidgets.Dropdown(options=list(airports.name),
                                     value=list(airports.name)[0],
                                     description='start:',
                                     disabled=False)

end_dropdown = ipywidgets.Dropdown(options=list(airports.name),
                                   value=list(airports.name)[1],
                                   description='end:',
                                   disabled=False)

# function
# calculate distance from a to b
# display map with ant path from a to b
def get_coordinates(a, b):
    start_lat = airports[airports.name == a].latitude.values[0]
    start_long = airports[airports.name == a].longitude.values[0]
    end_lat = airports[airports.name == b].latitude.values[0]
    end_long = airports[airports.name == b].longitude.values[0]
    start, end = (start_lat, start_long), (end_lat, end_long)
    
    m = Map(center=(38, -98), zoom=4, scroll_wheel_zoom=True)
    ant_path = AntPath(locations=[start, end])
    m.add_layer(ant_path)
    measure = MeasureControl(position='topleft', active_color='red', primary_length_unit='miles')
    m.add_control(measure)
    display(m)
    
    return distance.distance(start, end).miles

# interaction between function and widget
ipywidgets.interact(get_coordinates, a=start_dropdown, b=end_dropdown);

interactive(children=(Dropdown(description='start:', options=('Thigpen', 'Livingston Municipal', 'Meadow Lake'…